In [2]:
import pickle
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel

import warnings
warnings.filterwarnings("ignore")

import sys

### Import Data

In [3]:
# dataset = 'train'
dataset = 'train'
test = False

In [5]:
ern_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}.pkl")
ern_cov_fal_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}.pkl")
# ern_cov_peak_data_df = pd.read_pickle(f'data/models_pickles_new/ern_cov_peak_models_{dataset}.pkl')

crn_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}.pkl")
crn_cov_fal2_data_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}.pkl")
# crn_cov_fal6_data_df = pd.read_pickle(f"data/models_pickles_new/crn_cov_fal6_models_{dataset}.pkl")
# crn_cov_peak_data_df = pd.read_pickle(f'data/models_pickles_new/crn_cov_peak_models_{dataset}.pkl')

In [6]:
ern_data_df.shape

(260, 33)

In [45]:
datasets = [
    # ern_data_df, 
    ern_cov_fal_data_df, 
    # ern_cov_peak_data_df,
    # crn_data_df,
    crn_cov_fal2_data_df,
    # crn_cov_fal6_data_df,
    # crn_cov_peak_data_df,
]

### Create ERN - CRN measure

In [46]:
ern_values = ern_data_df['e_ERN'].to_numpy()
crn_values = crn_data_df['e_CRN'].to_numpy()

ern_crn_diff_values = ern_values - crn_values

In [48]:
ern_crn_diff_values.reshape(-1,1)
ern_crn_diff_df = pd.DataFrame(ern_crn_diff_values, columns=['ERN\CRN delta'])
ern_crn_diff_df.head()

,ERN\CRN delta
0,-2.285474e-07
1,-8.659172e-06
2,-7.245873e-06
3,-5.400762e-06
4,-5.829373e-06


Create delta ERN - CRN dataset

In [49]:
ern_cov_fal_data_df.columns

Index(['e_ERN', 'e_LT_F', 'Age', 'Sex', 'Handness', 'RRQ', 'DASS-21 Stress',
       'DASS-21 Dep', 'STAI-T', 'STAI-S Diff', 'BIS', 'BAS_D', 'BAS_PRZY',
       'BAS_NAG', 'WASH', 'OBSESS', 'HOARD', 'ORD', 'CHECK', 'NEU', 'OT',
       'WBSI', 'INDEC_F', 'PUN', 'REW', 'HARM', 'G_SE', 'T-CTR', 'OB_PERF',
       'PS', 'AMB', 'PRED', 'STAND', 'IUS-P', 'IUS-I', 'SES', 'performance'],
      dtype='object')

In [50]:
columns = ['Age', 'Sex', 'Handness', 'RRQ', 'DASS-21 Stress',
       'DASS-21 Dep', 'STAI-T', 'STAI-S Diff', 'BIS', 'BAS_D', 'BAS_PRZY',
       'BAS_NAG', 'WASH', 'OBSESS', 'HOARD', 'ORD', 'CHECK', 'NEU', 'OT',
       'WBSI', 'INDEC_F', 'PUN', 'REW', 'HARM', 'G_SE', 'T-CTR', 'OB_PERF',
       'PS', 'AMB', 'PRED', 'STAND', 'IUS-P', 'IUS-I', 'SES', 'performance']

data = ern_cov_fal_data_df[columns]

In [51]:
ern_crn_diff_data_df = pd.concat([ern_crn_diff_df, data], axis=1)
ern_crn_diff_data_df.head()

,ERN\CRN delta,Age,Sex,Handness,RRQ,DASS-21 Stress,DASS-21 Dep,STAI-T,STAI-S Diff,BIS,...,T-CTR,OB_PERF,PS,AMB,PRED,STAND,IUS-P,IUS-I,SES,performance
0,-2.285474e-07,0.424951,1.0,1.0,3.416667,1.285714,2.142857,2.25,10.298817,2.9,...,4.0,4.0,1.428571,4.83,4.25,4.666667,3.142857,2.8,2.4,1.684569
1,-8.659172e-06,0.424985,1.0,1.0,3.750000,1.142857,1.285714,2.00,8.185239,2.7,...,1.6,2.6,2.714286,4.17,3.75,5.000000,2.714286,1.8,3.1,1.376404
2,-7.245873e-06,0.425040,1.0,1.0,3.833333,1.857143,1.571429,2.40,14.306246,3.1,...,1.8,3.2,3.285714,4.00,4.25,4.333333,2.142857,3.0,2.4,-0.098470
3,-5.400762e-06,0.425113,1.0,1.0,2.250000,3.000000,2.571429,2.80,8.185239,3.0,...,1.2,2.0,3.428571,3.33,3.00,4.000000,3.428571,3.2,2.7,1.239223
4,-5.829373e-06,0.425062,1.0,1.0,3.916667,2.142857,1.571429,2.25,8.915005,3.0,...,2.0,1.8,2.285714,3.67,3.25,4.000000,2.857143,1.8,3.0,-0.746350


In [52]:
datasets.append(ern_crn_diff_data_df)

### Create correlation table

In [53]:
def corr_with_p(df):
    rho = df.corr()
    pval = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
    return rho.round(2).astype(str) + p, rho

In [54]:
corr_p_tables = []

for dataset in datasets:
    corr_p, corr = corr_with_p(dataset)
    corr_p_tables.append(corr_p)

In [55]:
for corr_table in corr_p_tables:
    display(corr_table)

,e_ERN,e_LT_F,Age,Sex,Handness,RRQ,DASS-21 Stress,DASS-21 Dep,STAI-T,STAI-S Diff,...,T-CTR,OB_PERF,PS,AMB,PRED,STAND,IUS-P,IUS-I,SES,performance
e_ERN,1.0***,-0.5***,-0.04,0.1,-0.04,-0.17***,-0.03,-0.04,-0.09,-0.03,...,-0.11*,-0.04,-0.05,-0.13**,-0.07,-0.09,-0.01,-0.08,0.07,-0.08
e_LT_F,-0.5***,1.0***,0.01,0.14**,-0.01,0.02,-0.07,0.01,-0.02,-0.02,...,0.08,0.0,0.05,0.02,-0.03,0.06,0.05,-0.01,-0.0,-0.01
Age,-0.04,0.01,1.0***,0.14**,-0.01,-0.07,-0.01,-0.03,-0.03,-0.05,...,-0.08,-0.05,0.05,-0.05,0.06,0.05,-0.0,0.04,0.03,0.05
Sex,0.1,0.14**,0.14**,1.0***,-0.11*,-0.06,-0.15**,-0.0,-0.13**,0.0,...,0.04,-0.15**,-0.05,-0.18***,-0.08,-0.09,0.04,0.02,0.04,0.06
Handness,-0.04,-0.01,-0.01,-0.11*,1.0***,-0.04,-0.03,-0.06,-0.09,-0.06,...,-0.02,-0.03,0.02,0.02,-0.03,0.06,-0.03,-0.05,0.11*,-0.03
RRQ,-0.17***,0.02,-0.07,-0.06,-0.04,1.0***,0.44***,0.4***,0.55***,0.03,...,0.21***,0.29***,0.07,0.36***,0.22***,0.13**,0.28***,0.41***,-0.47***,0.02
DASS-21 Stress,-0.03,-0.07,-0.01,-0.15**,-0.03,0.44***,1.0***,0.66***,0.61***,0.04,...,0.21***,0.3***,0.16***,0.29***,0.21***,0.09,0.27***,0.34***,-0.44***,-0.01
DASS-21 Dep,-0.04,0.01,-0.03,-0.0,-0.06,0.4***,0.66***,1.0***,0.69***,-0.03,...,0.25***,0.29***,-0.0,0.19***,0.09,0.02,0.17***,0.44***,-0.67***,0.07
STAI-T,-0.09,-0.02,-0.03,-0.13**,-0.09,0.55***,0.61***,0.69***,1.0***,0.05,...,0.26***,0.31***,0.03,0.35***,0.23***,0.06,0.29***,0.59***,-0.77***,0.04
STAI-S Diff,-0.03,-0.02,-0.05,0.0,-0.06,0.03,0.04,-0.03,0.05,1.0***,...,-0.08,0.0,0.03,0.09,0.1*,-0.06,0.1,0.04,-0.08,-0.02


,e_CRN,e_LT_F2_C,Age,Sex,Handness,RRQ,DASS-21 Stress,DASS-21 Dep,STAI-T,STAI-S Diff,...,T-CTR,OB_PERF,PS,AMB,PRED,STAND,IUS-P,IUS-I,SES,performance
e_CRN,1.0***,-0.38***,0.01,0.09,-0.02,-0.04,-0.03,-0.04,-0.04,0.03,...,-0.11*,-0.15**,-0.12**,-0.06,-0.04,-0.07,0.01,-0.01,0.0,0.02
e_LT_F2_C,-0.38***,1.0***,0.01,0.04,-0.01,0.09,0.01,0.02,0.03,0.06,...,0.01,0.13**,0.02,-0.01,-0.04,0.06,-0.04,0.04,-0.03,0.1
Age,0.01,0.01,1.0***,0.14**,-0.01,-0.07,-0.01,-0.03,-0.03,-0.05,...,-0.08,-0.05,0.05,-0.05,0.06,0.05,-0.0,0.04,0.03,0.05
Sex,0.09,0.04,0.14**,1.0***,-0.11*,-0.06,-0.15**,-0.0,-0.13**,0.0,...,0.04,-0.15**,-0.05,-0.18***,-0.08,-0.09,0.04,0.02,0.04,0.06
Handness,-0.02,-0.01,-0.01,-0.11*,1.0***,-0.04,-0.03,-0.06,-0.09,-0.06,...,-0.02,-0.03,0.02,0.02,-0.03,0.06,-0.03,-0.05,0.11*,-0.03
RRQ,-0.04,0.09,-0.07,-0.06,-0.04,1.0***,0.44***,0.4***,0.55***,0.03,...,0.21***,0.29***,0.07,0.36***,0.22***,0.13**,0.28***,0.41***,-0.47***,0.02
DASS-21 Stress,-0.03,0.01,-0.01,-0.15**,-0.03,0.44***,1.0***,0.66***,0.61***,0.04,...,0.21***,0.3***,0.16***,0.29***,0.21***,0.09,0.27***,0.34***,-0.44***,-0.01
DASS-21 Dep,-0.04,0.02,-0.03,-0.0,-0.06,0.4***,0.66***,1.0***,0.69***,-0.03,...,0.25***,0.29***,-0.0,0.19***,0.09,0.02,0.17***,0.44***,-0.67***,0.07
STAI-T,-0.04,0.03,-0.03,-0.13**,-0.09,0.55***,0.61***,0.69***,1.0***,0.05,...,0.26***,0.31***,0.03,0.35***,0.23***,0.06,0.29***,0.59***,-0.77***,0.04
STAI-S Diff,0.03,0.06,-0.05,0.0,-0.06,0.03,0.04,-0.03,0.05,1.0***,...,-0.08,0.0,0.03,0.09,0.1*,-0.06,0.1,0.04,-0.08,-0.02


,ERN\CRN delta,Age,Sex,Handness,RRQ,DASS-21 Stress,DASS-21 Dep,STAI-T,STAI-S Diff,BIS,...,T-CTR,OB_PERF,PS,AMB,PRED,STAND,IUS-P,IUS-I,SES,performance
ERN\CRN delta,1.0***,-0.05,0.02,-0.03,-0.15**,-0.01,0.0,-0.07,-0.07,-0.06,...,-0.01,0.11*,0.07,-0.08,-0.04,-0.04,-0.02,-0.08,0.08,-0.11*
Age,-0.05,1.0***,0.14**,-0.01,-0.07,-0.01,-0.03,-0.03,-0.05,-0.05,...,-0.08,-0.05,0.05,-0.05,0.06,0.05,-0.0,0.04,0.03,0.05
Sex,0.02,0.14**,1.0***,-0.11*,-0.06,-0.15**,-0.0,-0.13**,0.0,-0.24***,...,0.04,-0.15**,-0.05,-0.18***,-0.08,-0.09,0.04,0.02,0.04,0.06
Handness,-0.03,-0.01,-0.11*,1.0***,-0.04,-0.03,-0.06,-0.09,-0.06,-0.0,...,-0.02,-0.03,0.02,0.02,-0.03,0.06,-0.03,-0.05,0.11*,-0.03
RRQ,-0.15**,-0.07,-0.06,-0.04,1.0***,0.44***,0.4***,0.55***,0.03,0.52***,...,0.21***,0.29***,0.07,0.36***,0.22***,0.13**,0.28***,0.41***,-0.47***,0.02
DASS-21 Stress,-0.01,-0.01,-0.15**,-0.03,0.44***,1.0***,0.66***,0.61***,0.04,0.49***,...,0.21***,0.3***,0.16***,0.29***,0.21***,0.09,0.27***,0.34***,-0.44***,-0.01
DASS-21 Dep,0.0,-0.03,-0.0,-0.06,0.4***,0.66***,1.0***,0.69***,-0.03,0.38***,...,0.25***,0.29***,-0.0,0.19***,0.09,0.02,0.17***,0.44***,-0.67***,0.07
STAI-T,-0.07,-0.03,-0.13**,-0.09,0.55***,0.61***,0.69***,1.0***,0.05,0.64***,...,0.26***,0.31***,0.03,0.35***,0.23***,0.06,0.29***,0.59***,-0.77***,0.04
STAI-S Diff,-0.07,-0.05,0.0,-0.06,0.03,0.04,-0.03,0.05,1.0***,0.18***,...,-0.08,0.0,0.03,0.09,0.1*,-0.06,0.1,0.04,-0.08,-0.02
BIS,-0.06,-0.05,-0.24***,-0.0,0.52***,0.49***,0.38***,0.64***,0.18***,1.0***,...,0.13**,0.26***,-0.02,0.45***,0.32***,0.01,0.32***,0.47***,-0.55***,-0.04
